---
# Convert Radar Gauge
---

In [1]:
import os
from multiprocessing import Process
import datetime
import subprocess
import xarray as xr

from src.util import fix_ds

In [2]:
def bin2nc(dt, wgrib2='/home/yoshimi/grib2/wgrib2/wgrib2',
           save_path='/media/yoshimi/9E9401BB94019745/pixel/garadar'):
    # bin file
    bin_file = dt.strftime('/media/yoshimi/9E9401BB94019745/JMA/DATA/%Y/%m/%d/Z__C_RJTD_%Y%m%d%H%M%S_SRF_GPV_Ggis1km_Prr60lv_ANAL_grib2.bin')
    if not os.path.isfile(bin_file):
        return
    # convert
    date = dt.strftime('%Y_%m_%d_%H_%M')
    nc_file = f'{save_path}/{date}.nc'
    subprocess.run([wgrib2, bin_file, '-netcdf', nc_file])
    
    # compress
    ds = xr.load_dataset(nc_file)
    ds = fix_ds(ds)
    comp = dict(zlib=True, complevel=9)
    encoding = {var: comp for var in ds.data_vars}
    ds.to_netcdf(nc_file, encoding=encoding)

In [3]:
def convert_garadar(start, end, num='30', timescale='minutes', 
                    wgrib2='/home/yoshimi/grib2/wgrib2/wgrib2',
                    save_path='/media/yoshimi/9E9401BB94019745/pixel/garadar'):
    dt = datetime.datetime.strptime(start, '%Y/%m/%d %H:%M:%S')
    dt_limit = datetime.datetime.strptime(end, '%Y/%m/%d %H:%M:%S')
    
    while dt < dt_limit:
        bin2nc(dt, wgrib2=wgrib2, save_path=save_path)
        dt = dt + eval('datetime.timedelta(' + timescale + '=' + num + ')')

In [4]:
dates = ['2010/01/01 00:00:00', 
         '2011/01/01 00:00:00', 
         '2012/01/01 00:00:00', 
         '2013/01/01 00:00:00', 
         '2014/01/01 00:00:00', 
         '2015/01/01 00:00:00', 
         '2016/01/01 00:00:00', 
         '2017/01/01 00:00:00', 
         '2018/01/01 00:00:00', 
         '2019/01/01 00:00:00', 
         '2020/01/01 00:00:00', 
         '2021/01/01 00:00:00']
process_num = len(dates) - 1 

In [ ]:
%%capture
process_list = []
for i in range(process_num):
    process = Process(
        target=convert_garadar,
        kwargs={'start':dates[i],
                'end':dates[i+1]})
    
    process.start()
    process_list.append(process)

for process in process_list:
    process.join()